In [1]:
# Import libraries
import pandas as pd
import statsmodels as sm
import numpy as np
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
import matplotlib.pyplot as plt
from scipy import stats

In [6]:
# Upload

datafile = "/Users/justinpassek/Downloads/7443_MM_TSG_Dynamic_SPV_Report_20191212_162901_2744311767.csv"
df = pd.read_csv(datafile,
                 #skiprows=11
                )

#print(df)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,12,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
# Preprocess Data
# Change Column Names
new_names = {
           'Sprint Consumer : Sprint_Conversion_PV : Consumer Shop Page View_2018: Total Conversions':'SPV',
            'Feed 3 - Reporting dimension 3 value':'Gradient',
            'Feed 3 - Reporting dimension 4 value':'Lightwave',
            'Feed 3 - Reporting dimension 5 value':'Messaging_Variation',
            'Feed 3 - Reporting dimension 6 value':'CTA'
            }

df.rename(columns=new_names,inplace=True)

#10 Impression Minimum
df = df[df["Impressions"]>10]

#There Must be More Conversions than Impressions
#df = df[df["Impressions"]>df["Clicks"]]

#Remove Unidentifiable Values
df = df[df.Gradient != '---']
df = df[df.Lightwave != '---']
df = df[df.Messaging_Variation != '---']
df = df[df.CTA != '---']

#Add CTR
#df['CTR'] = df['Clicks']/df['Impressions']

#Add SPV_Rate
df['SPV_Rate'] = df['SPV']/df['Impressions']
print(df)

                              Advertiser  \
0      Sprint - Sprint.com - Mighty Hive   
1      Sprint - Sprint.com - Mighty Hive   
2      Sprint - Sprint.com - Mighty Hive   
3      Sprint - Sprint.com - Mighty Hive   
4      Sprint - Sprint.com - Mighty Hive   
...                                  ...   
79428  Sprint - Sprint.com - Mighty Hive   
79431  Sprint - Sprint.com - Mighty Hive   
79439  Sprint - Sprint.com - Mighty Hive   
79448  Sprint - Sprint.com - Mighty Hive   
79449  Sprint - Sprint.com - Mighty Hive   

                                 Dynamic Element 1  \
0      Sprint_TSG_DynamicFeed_Schema_Tracking_Feed   
1      Sprint_TSG_DynamicFeed_Schema_Tracking_Feed   
2      Sprint_TSG_DynamicFeed_Schema_Tracking_Feed   
3      Sprint_TSG_DynamicFeed_Schema_Tracking_Feed   
4      Sprint_TSG_DynamicFeed_Schema_Tracking_Feed   
...                                            ...   
79428  Sprint_TSG_DynamicFeed_Schema_Tracking_Feed   
79431  Sprint_TSG_DynamicFeed_Schema_Tr

In [8]:
#Narrow to Relevant Columns
imp_columns = [
                "Gradient"
               ,"Lightwave"
               ,"Messaging_Variation"
               ,"CTA"
               ,"Impressions"
               ,"Clicks"
#               ,"CTR"
               ,"SPV"
               ,"SPV_Rate"
              ]
df = df[imp_columns]

In [17]:
#PV Function
def pivot_tbl_4_dim(x,y,z,a):
    pvdf = df.groupby([x,y,z,a],as_index=False)[('Impressions','SPV')].sum()
#    pvdf['CTR'] = pvdf.Clicks/pvdf.Impressions
    pvdf['SPV_Rate'] = pvdf.SPV/pvdf.Impressions
    return pvdf

print(pivot_tbl_4_dim(df.Gradient,df.Lightwave,df.Messaging_Variation,df.CTA))

pivot_tbl_4_dim(df.Gradient,df.Lightwave,df.Messaging_Variation,df.CTA).to_csv("/Users/justinpassek/Downloads/Sprint_Testing_Components.csv", index=False)

          Gradient  Lightwave Messaging_Variation              CTA  \
0      black-black  lightwave   discover-why-bill       Switch Now   
1      black-black  lightwave  discover-why-money  Make the Switch   
2      black-black  lightwave  discover-why-money       Switch Now   
3      black-black  lightwave        see-why-bill       Learn More   
4      black-black  lightwave        see-why-bill  Make the Switch   
5      black-black  lightwave        see-why-bill       Switch Now   
6      black-black  lightwave       see-why-money       Learn More   
7      black-black  lightwave       see-why-money       Switch Now   
8      black-black       none   discover-why-bill       Switch Now   
9      black-black       none  discover-why-money       Switch Now   
10     black-black       none        see-why-bill       Learn More   
11     black-black       none        see-why-bill  Make the Switch   
12     black-black       none        see-why-bill       Switch Now   
13     black-black  

In [10]:
#PV Function
def pivot_tbl_2_dim(x,y):
    pvdf = df.groupby([x,y],as_index=False)[('Impressions','SPV')].sum()
#    pvdf['CTR'] = pvdf.Clicks/pvdf.Impressions
    pvdf['SPV_Rate'] = pvdf.SPV/pvdf.Impressions
    return pvdf

print(pivot_tbl_2_dim(df.Gradient,df.Lightwave))

def two_way_anova_test(x,y):
    SPV_Rate = sum(df.SPV)/sum(df.Impressions)
    formula = 'SPV ~ x + y + x:y'
    model = ols(formula,data=df).fit()
    aov_table = anova_lm(model, typ=1)
    return aov_table

print(two_way_anova_test(df.Gradient,df.Lightwave))

         Gradient  Lightwave  Impressions    SPV  SPV_Rate
0     black-black  lightwave     33692317  88907  0.002639
1     black-black       none     30936506  82104  0.002654
2  black-darkblue  lightwave      5496048  10273  0.001869
3  black-darkblue       none          626      0  0.000000
4  black-darkgold  lightwave     18643575  60546  0.003248
5  black-darkgold       none     15000415  55734  0.003715
               df        sum_sq       mean_sq          F        PR(>F)
x             2.0  3.104837e+04  15524.185636  19.793184  2.551003e-09
y             1.0  8.761991e+02    876.199108   1.117145  2.905382e-01
x:y           2.0  8.254920e+02    412.746001   0.526247  5.908208e-01
Residual  61148.0  4.795959e+07    784.319795        NaN           NaN


In [11]:
print(pivot_tbl_2_dim(df.Messaging_Variation,df.CTA))
print(two_way_anova_test(df.Messaging_Variation,df.CTA))

   Messaging_Variation              CTA  Impressions     SPV  SPV_Rate
0    discover-why-bill       Learn More         1591       1  0.000629
1    discover-why-bill  Make the Switch         1743       0  0.000000
2    discover-why-bill       Switch Now      8282253   15515  0.001873
3   discover-why-money  Make the Switch         1684       0  0.000000
4   discover-why-money       Switch Now      9830789   19963  0.002031
5         see-why-bill       Learn More      7355347   15077  0.002050
6         see-why-bill  Make the Switch         4793      24  0.005007
7         see-why-bill       Switch Now     23243924   46700  0.002009
8        see-why-money       Learn More     15895074   30464  0.001917
9        see-why-money  Make the Switch         1773      36  0.020305
10       see-why-money       Switch Now     39150516  169784  0.004337
               df        sum_sq        mean_sq           F         PR(>F)
x             3.0  4.916459e+05  163881.965987  214.552418  1.848540e-138


In [12]:
print(pivot_tbl_2_dim(df.Messaging_Variation,df.Gradient))
print(two_way_anova_test(df.Messaging_Variation,df.Gradient))

  Messaging_Variation        Gradient  Impressions     SPV  SPV_Rate
0   discover-why-bill     black-black      5704292   10709  0.001877
1   discover-why-bill  black-darkgold      2581295    4807  0.001862
2  discover-why-money     black-black      4926939   10454  0.002122
3  discover-why-money  black-darkgold      4905534    9509  0.001938
4        see-why-bill     black-black     15280956   31149  0.002038
5        see-why-bill  black-darkblue      3047010    5658  0.001857
6        see-why-bill  black-darkgold     12276098   24994  0.002036
7       see-why-money     black-black     38716636  118699  0.003066
8       see-why-money  black-darkblue      2449664    4615  0.001884
9       see-why-money  black-darkgold     13881063   76970  0.005545
               df        sum_sq        mean_sq           F         PR(>F)
x             3.0  4.916459e+05  163881.965987  212.430951  4.289178e-137
y             2.0  1.004486e+05   50224.282167   65.102905   5.704656e-29
x:y           6.0  

In [13]:
print(pivot_tbl_2_dim(df.Messaging_Variation,df.Lightwave))
print(two_way_anova_test(df.Messaging_Variation,df.Lightwave))

  Messaging_Variation  Lightwave  Impressions     SPV  SPV_Rate
0   discover-why-bill  lightwave      3002707    5575  0.001857
1   discover-why-bill       none      5282880    9941  0.001882
2  discover-why-money  lightwave      4759388    9683  0.002035
3  discover-why-money       none      5073085   10280  0.002026
4        see-why-bill  lightwave     17978333   36475  0.002029
5        see-why-bill       none     12625731   25326  0.002006
6       see-why-money  lightwave     32091512  107993  0.003365
7       see-why-money       none     22955851   92291  0.004020
               df        sum_sq        mean_sq           F         PR(>F)
x             3.0  4.916459e+05  163881.965987  211.165167  2.800108e-136
y             1.0  1.621395e+04   16213.952001   20.891999   4.868694e-06
x:y           3.0  3.002781e+04   10009.269599   12.897143   2.029336e-08
Residual  61146.0  4.745445e+07     776.084277         NaN            NaN


In [14]:
print(pivot_tbl_2_dim(df.CTA,df.Gradient))
print(two_way_anova_test(df.CTA,df.Gradient))

               CTA        Gradient  Impressions     SPV  SPV_Rate
0       Learn More     black-black     15425018   31304  0.002029
1       Learn More  black-darkblue      2449664    4615  0.001884
2       Learn More  black-darkgold      5377330    9623  0.001790
3  Make the Switch     black-black         6759      59  0.008729
4  Make the Switch  black-darkgold         3234       1  0.000309
5       Switch Now     black-black     49197046  139648  0.002839
6       Switch Now  black-darkblue      3047010    5658  0.001857
7       Switch Now  black-darkgold     28263426  106656  0.003774
               df        sum_sq       mean_sq          F        PR(>F)
x             2.0  1.391443e+05  69572.126405  88.938449  2.695228e-39
y             2.0  1.321113e+04   6605.564374   8.444311  2.153718e-04
x:y           4.0  8.512626e+03   2128.156612   2.720557  2.792930e-02
Residual  61146.0  4.783148e+07    782.250281        NaN           NaN


In [15]:
print(pivot_tbl_2_dim(df.Lightwave,df.CTA))
print(two_way_anova_test(df.Lightwave,df.CTA))

   Lightwave              CTA  Impressions     SPV  SPV_Rate
0  lightwave       Learn More     15039413   30266  0.002012
1  lightwave  Make the Switch         4999      23  0.004601
2  lightwave       Switch Now     42787528  129437  0.003025
3       none       Learn More      8212599   15276  0.001860
4       none  Make the Switch         4994      37  0.007409
5       none       Switch Now     37719954  122525  0.003248
               df        sum_sq       mean_sq          F        PR(>F)
x             1.0  3.608365e+03   3608.364943   4.610886  3.177353e-02
y             2.0  1.357253e+05  67862.672182  86.717135  2.469077e-38
x:y           2.0  1.000042e+02     50.002090   0.063894  9.381042e-01
Residual  61148.0  4.785290e+07    782.575115        NaN           NaN


In [16]:
def four_way_anova_test(x,y,z,a):
    #CTR = sum(df.Clicks)/sum(df.Impressions)
    SPV_Rate = sum(df.SPV)/sum(df.Impressions)
    formula = 'SPV ~ x + y + z + a'
    model = ols(formula,data=df).fit()
    aov_table = anova_lm(model, typ=1)
    return aov_table

print(four_way_anova_test(df.Gradient,df.Lightwave, df.Messaging_Variation, df.CTA))

               df        sum_sq        mean_sq           F         PR(>F)
x             2.0  3.104837e+04   15524.185636   20.206934   1.687093e-09
y             1.0  8.761991e+02     876.199108    1.140498   2.855514e-01
z             3.0  5.694135e+05  189804.490353  247.057519  2.285226e-159
a             2.0  4.157212e+05  207860.585954  270.560094  1.033080e-117
Residual  61145.0  4.697528e+07     768.260326         NaN            NaN


In [ ]:
Linear model one w/ features that have sig p value based on anova
pull coefficients and do analysis on individual components
In this instance, Gradient, Messaging Variation, CTA are the relevant variables